In [ ]:
import pandas as pd
import time
from ortools.linear_solver import pywraplp

In [ ]:
AllData = pd.ExcelFile("Input Data.xlsx")
StudentCourseData = pd.read_excel(AllData, 'StudentCourse')
TeacherCourseData = pd.read_excel(AllData, 'TeacherCourse')
TeacherBlockData = pd.read_excel(AllData, 'TeacherBlock')
CourseBlockData = pd.read_excel(AllData, 'CourseBlock')

print(StudentCourseData)
print("")
print(TeacherCourseData)
print("")
print(TeacherBlockData)
print("")
print(CourseBlockData)

In [ ]:
num_students = StudentCourseData.shape[0]
num_teachers = TeacherCourseData.shape[0]
num_courses = TeacherCourseData.shape[1]-1
num_blocks = TeacherBlockData.shape[1]-1
all_students = range(num_students)
all_teachers = range(num_teachers)
all_courses = range(num_courses)
all_blocks = range(num_blocks)
StudentList=StudentCourseData["Student"]
TeacherList=TeacherCourseData["Teacher"]
CourseList=CourseBlockData["Course"]
BlockList=list(CourseBlockData)[1:5]

In [ ]:
StudentCourseMatrix = [[-5 for c in all_courses] for s in all_students]
for c in range(1,num_courses+1):
    for s in all_students:
        if StudentCourseData[list(StudentCourseData)[c]][s]=='Y':
            StudentCourseMatrix[s][c-1]=2
            
TeacherCourseMatrix = [[-100 for c in all_courses] for t in all_teachers]
for c in range(1,num_courses+1):
    for t in all_teachers:
        if TeacherCourseData[list(TeacherCourseData)[c]][t]=='Y':
            TeacherCourseMatrix[t][c-1]=5

TeacherBlockMatrix = [[0 for b in all_blocks] for t in all_teachers]
for b in range(1,num_blocks+1):
    for t in all_teachers:
        if TeacherBlockData[list(TeacherBlockData)[b]][t]=='N':
            TeacherBlockMatrix[t][b-1]=-1

CourseBlockMatrix = [[0 for b in all_blocks] for c in all_courses]
for b in range(1,num_blocks+1):
    for c in all_courses:
        if CourseBlockData[list(CourseBlockData)[b]][c]=='N':
            CourseBlockMatrix[c][b-1]=-1

In [ ]:
solver = pywraplp.Solver('St. Margarets School', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

start_time = time.time()

# Define our binary variables for the teachers
X = {}
for t in all_teachers:
    for c in all_courses:
        for b in all_blocks:
            X[t,c,b] = solver.BoolVar('X[%i,%i,%i]' % (t, c, b))

# Define our binary variables for the students
Y = {}
for s in all_students:
    for c in all_courses:
        for b in all_blocks:
            Y[s,c,b] = solver.BoolVar('Y[%i,%i,%i]' % (s, c, b))
            
# Define our Suitability Coefficients for the teachers
Suitability = [[[0 for b in all_blocks] for c in all_courses] for t in all_teachers]
for t in all_teachers:
    for c in all_courses:
        for b in all_blocks:
            Suitability[t][c][b]=TeacherCourseMatrix[t][c]+TeacherBlockMatrix[t][b]+CourseBlockMatrix[c][b]

# Define our Preference Coefficients for the students
Preference = [[[0 for b in all_blocks] for c in all_courses] for s in all_students]
for s in all_students:
    for c in all_courses:
        for b in all_blocks:
            Preference[s][c][b]=StudentCourseMatrix[s][c]

# Define our objective function
solver.Maximize(solver.Sum([Suitability[t][c][b] * X[t,c,b] for t in all_teachers for c in all_courses for b in all_blocks])
               +solver.Sum([Preference[s][c][b] * Y[s,c,b] for s in all_students for c in all_courses for b in all_blocks])               )

# Each teacher teaches exactly two courses
for t in all_teachers:
    solver.Add(solver.Sum([X[t,c,b] for c in all_courses for b in all_blocks]) <= 2)

# Each teacher teaches at most once in any slot
for t in all_teachers:
    for b in all_blocks:
        solver.Add(solver.Sum([X[t,c,b] for c in all_courses]) <= 1)

# Each course is covered in one block by one teacher
for c in all_courses:
    solver.Add(solver.Sum([X[t,c,b] for t in all_teachers for b in all_blocks]) == 1)  

# Each block has exactly three courses
for b in all_blocks:
    solver.Add(solver.Sum([X[t,c,b] for t in all_teachers for c in all_courses]) == 3)  

# Each student must take one course each block
for s in all_students:
    for b in all_blocks:
        solver.Add(solver.Sum([Y[s,c,b] for c in all_courses]) == 1)  
    
# No student can take the same course twice
for s in all_students:
    for c in all_courses:
        solver.Add(solver.Sum([Y[s,c,b] for b in all_blocks]) <= 1)  

# No student can take a course in a block when that course is not offered
for c in all_courses:
    for b in all_blocks:
        for s in all_students:
            solver.Add(Y[s,c,b] <= solver.Sum([X[t,c,b] for t in all_teachers]))  

current_time = time.time() 
reading_time = current_time - start_time         
sol = solver.Solve()
solving_time = time.time() - current_time

print('Optimization Complete with Total Happiness Score of', round(solver.Objective().Value()))
print("")
print('Our program needed', round(reading_time,3), 'seconds to read the data and', 
      round(solving_time,3), 'seconds to determine the optimal solution')
                

In [ ]:
# Format Output for both Teachers and Students

TeacherCourseOutput = TeacherCourseData.copy()

for c in CourseList:
    for t in all_teachers:
        TeacherCourseOutput[c][t]="-"

for t in all_teachers:
    for c in all_courses:
        for b in all_blocks:
            if X[t,c,b].solution_value() > 0:
                TeacherCourseOutput[CourseList[c]][t]=BlockList[b]

StudentCourseOutput = StudentCourseData.copy()

for c in CourseList:
    for s in all_students:
        StudentCourseOutput[c][s]="-"

for s in all_students:
    for c in all_courses:
        for b in all_blocks:
            if Y[s,c,b].solution_value() > 0:
                StudentCourseOutput[CourseList[c]][s]=BlockList[b]


In [ ]:
TeacherCourseOutput

In [ ]:
StudentCourseOutput